## Fine-Tuning and Deploying Custom Models on Amazon Bedrock

### Introduction:

This notebook demonstrates the process of fine-tuning a Cohere `command-light-text-v14` model on Amazon Bedrock for generating concise clinical notes from patient-doctor conversations. The key steps include resource setup, data preparation, fine-tuning, and deployment. For a detailed explanation of the fine-tuning process, including the use case and configurations, please refer to the accompanying article: [Fine-Tuning and Deploying Custom AI Models on Amazon Bedrock: A Practical Guide](https://dev.to/miladrezaei/fine-tuning-and-deploying-custom-ai-models-on-amazon-bedrock-a-practical-guide-39m6).

The dataset used in this notebook is a **combined version of the ACI-Bench training and validation datasets**:
- **Source**: [ACI-Bench: Ambient Clinical Intelligence Benchmark](http://github.com/microsoft/clinical_visit_note_summarization_corpus).
- **Contents**: Patient-doctor dialogues paired with concise clinical notes summarizing the interaction.

The combined dataset contains **88 examples**, which is intentionally small for this demonstration.

### Why a Smaller Dataset?
For the purpose of this guideline, I have chosen a smaller dataset to:
- **Reduce the cost of fine-tuning**: Fine-tuning on large datasets can be computationally expensive. Using a small dataset minimizes resource consumption, making it practical for a demonstration.
- **Simplify the workflow**: A smaller dataset allows us to focus on the technical steps involved in fine-tuning without the overhead of managing large-scale data.

However, in a real-world scenario, **fine-tuning tasks typically require thousands or tens of thousands of examples** to achieve meaningful results. Future steps may include:
- Expanding the dataset with additional clinical dialogues.
- Augmenting the data using paraphrasing, back-translation, or similar techniques.

### Use Case
The fine-tuned model will be trained to:
- Understand the context of medical dialogues.
- Generate concise, structured clinical notes summarizing key patient symptoms and doctor recommendations.

This use case is particularly relevant for:
- **Healthcare providers**: Automating medical documentation and reducing administrative overhead.
- **Telemedicine**: Summarizing virtual consultations for patient records.
- **Clinical NLP**: Building AI systems for clinical note generation.

### Dataset Preparation
1. **Combination**:
   - The original ACI-Bench training (`train.csv`) and validation (`valid.csv`) datasets were combined to create a single dataset of 88 examples.
2. **Preprocessing**:
   - The dialogues were used as **prompts**, and their corresponding notes were used as **completions** to create input-output pairs.
3. **Format**:
   - The processed dataset is saved in JSONL format to prepare it for fine-tuning on Amazon Bedrock.

## Technical Prerequisites

- Access to Amazon Bedrock with Cohere command-light-text-v14 model enabled
- AWS SDK (boto3) configured with appropriate IAM permissions
- Python environment with required dependencies

## Step 1: Install Required Libraries

We begin by installing and importing the required libraries for handling data processing, AWS interactions, and API calls:

In [ ]:
!pip install datasets boto3 pandas json

import pandas as pd
import boto3
import json
import requests
import os

## Step 2: Data Preparation

In this step, we load the combined dataset from a CSV file to inspect its structure and size before formatting it for fine-tuning.

- **Dataset Source**: The dataset is a consolidated version of training and validation data from ACI-Bench.
- **Objective**: The dataset contains doctor-patient dialogues (`dialogue`) and corresponding clinical notes (`note`). These will be reformatted into prompt-completion pairs for fine-tuning.
- **Validation**: By checking the dataset size and column names, we ensure that the data is correctly loaded and aligned with the required format for downstream tasks.

This validation step is critical to avoid errors during processing and to confirm that the dataset matches the expected schema.

In [ ]:
# Define dataset splits and path
train_dataset_path = "dataset/data.csv"

train_dataset = pd.read_csv(train_dataset_path)

# Check the dataset size and structure
print(f"Total records in the dataset: {len(train_dataset)}")
print(f"Columns in the dataset: {train_dataset.columns.tolist()}")

Once the dataset is loaded and validated, we format it into the JSONL structure required for fine-tuning. This involves the following steps:

1. **Defining the Output Path**:
   - The reformatted dataset will be saved as a JSONL file in the `dataset` directory.
   - JSONL (JSON Lines) format is preferred for fine-tuning as it allows efficient storage of individual prompt-completion pairs.

2. **Formatting Data**:
   - Each row in the dataset is converted into a dictionary with:
     - `prompt`: Contains the patient-doctor dialogue prefixed with the instruction: "Summarize the following conversation."
     - `completion`: Contains the corresponding clinical note summarizing the dialogue.
   - This structure aligns with the format expected by fine-tuning frameworks like Amazon Bedrock.



In [ ]:
# Define output path for JSONL
output_file_name = 'clinical_notes_fine_tune.jsonl'
output_file_path = os.path.join('dataset', output_file_name)
output_dir = os.path.dirname(output_file_path)

# Prepare and save the dataset in the fine-tuning JSONL format
with open(output_file_path, 'w') as outfile:
    for _, row in train_dataset.iterrows():
        formatted_entry = {
            "completion": row['note'],  # Replace 'note' with the correct column name
            "prompt": f"Summarize the following conversation.\n\n{row['dialogue']}"  # Replace 'dialogue' as needed
        }
        json.dump(formatted_entry, outfile)
        outfile.write('\n')
    print(f"Dataset has been reformatted and saved to {output_file_path}.")

# Optional: Print example formatted entry for debugging
example_formatted_entry = {
    "completion": train_dataset.iloc[0]['note'],  # Replace with the correct column name
    "prompt": f"Summarize the following conversation.\n\n{train_dataset.iloc[0]['dialogue']}"  # Replace as needed
}
print("Example formatted entry:")
print(json.dumps(example_formatted_entry, indent=4))

### Step 3: Upload the Dataset to S3

This step uploads the prepared JSONL dataset to an S3 bucket, making it accessible for fine-tuning with Amazon Bedrock.

1. **Bucket Setup**:
   - Checks if the specified S3 bucket (`bucket_name`) exists. If not, it creates the bucket in the specified AWS region (`region`).

2. **File Upload**:
   - The dataset is uploaded to the S3 bucket at the specified key (`s3_key`).
   - The final S3 location will be `s3://<bucket_name>/<s3_key>`.

### Notes:
- Ensure your AWS region and credentials are configured correctly.
- Update `bucket_name` and `region` as needed.

In [ ]:
# Define the file path and S3 details
bucket_name = 'bedrock-finetuning-bucket25112024' # This bucket has to be created 
s3_key = output_file_name

# Specify the region
region = 'us-east-1'  # Change this if needed

# Initialize S3 client with the specified region
s3_client = boto3.client('s3', region_name=region)

# Check if the bucket exists
try:
    existing_buckets = s3_client.list_buckets()
    bucket_exists = any(bucket['Name'] == bucket_name for bucket in existing_buckets['Buckets'])

    if not bucket_exists:
        # Create the bucket based on the region
        try:
            if bucket_region == 'us-east-1':
                # For us-east-1, do not specify LocationConstraint
                s3_client.create_bucket(Bucket=bucket_name)
                print(f"Bucket {bucket_name} created successfully in us-east-1.")
            else:
                # For other regions, specify the LocationConstraint
                s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': bucket_region}
                )
                print(f"Bucket {bucket_name} created successfully in {bucket_region}.")
        except Exception as e:
            print(f"Error creating bucket: {e}")
            raise e
    else:
        print(f"Bucket {bucket_name} already exists.")

    # Upload the file to S3
    try:
        s3_client.upload_file(output_file_path, bucket_name, s3_key)
        print(f"File uploaded to s3://{bucket_name}/{s3_key}")
    except Exception as e:
        print(f"Error uploading to S3: {e}")

except Exception as e:
    print(f"Error: {e}")

## Step 5: Fine-Tune the Model on Amazon Bedrock

This step initializes the Amazon Bedrock client and retrieves a list of foundation models available for fine-tuning. The Bedrock client is set up for the specified AWS region, ensuring compatibility with the service. Using the `list_foundation_models` method, the script identifies models that support fine-tuning and displays their details, such as `modelId` and `modelName`. This information helps in selecting the appropriate model for the fine-tuning process.

Ensure the AWS region is correctly configured and that you have the necessary permissions to access Bedrock services.

In [ ]:
# Specify the AWS region and initialize the Bedrock client
bedrock_region = "us-east-1"

# Initialize the Bedrock client
bedrock = boto3.client(service_name="bedrock", region_name=bedrock_region)

# List foundation models available for fine-tuning
response = bedrock.list_foundation_models(byCustomizationType="FINE_TUNING")

# Display the available models
for model in response["modelSummaries"]:
    print(f"Model ID: {model['modelId']}")
    print(f"Model Name: {model['modelName']}")
    print("-----")

## Step 6: Create an IAM Role for Fine-Tuning

This step sets up an IAM role with the required permissions to facilitate the fine-tuning process on Amazon Bedrock. The role is configured with a trust policy, allowing the Bedrock service to assume the role, and permissions to access the S3 bucket where the fine-tuning dataset is stored.

- **Role Definition**:
  - The role is created with a trust policy specifying `bedrock.amazonaws.com` as the trusted service. This allows Bedrock to use the role for the fine-tuning process.

- **Permissions**:
  - A custom policy is attached to the role, granting it access to perform `s3:GetObject`, `s3:PutObject`, and `s3:ListBucket` actions on the specified S3 bucket.

- **Result**:
  - The role's ARN (`role_arn`) is displayed upon successful creation, which can then be used to configure the fine-tuning job.

### Notes:
- Replace `bucket_name` with the name of your S3 bucket containing the dataset.
- Ensure your AWS credentials have sufficient permissions to create IAM roles and policies.

In [ ]:
# Initialize IAM client
iam_client = boto3.client('iam')

# Define role name and trust policy
role_name = "BedrockFineTuningRole" # Customizable: Name of the IAM role
role_description = "Role for Bedrock fine-tuning job" # Customizable: Description of the IAM role

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
        }
    ]
}

# Create the IAM role
try:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description=role_description
    )
    role_arn = response['Role']['Arn']
    print(f"Created role with ARN: {role_arn}")
except Exception as e:
    print(f"Error creating role: {e}")

# Attach permission policies to allow access to S3
permission_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}",
                f"arn:aws:s3:::{bucket_name}/*"
            ]
        }
    ]
}

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName="BedrockFineTuningS3Policy",
    PolicyDocument=json.dumps(permission_policy)
)

## Step 7: Submit the Fine-Tuning Job

- **Job Parameters**:
  - `base_model_id`: The identifier for the foundation model being fine-tuned. Here, it's set to `cohere.command-light-text-v14:7:4k`, which is suitable for summarization tasks.
  - `job_name`: A unique name for the fine-tuning job, useful for tracking.
  - `model_name`: The name of the custom fine-tuned model that will be created.

- **IAM Role**:
  - `roleArn`: The ARN of the IAM role created earlier, allowing Bedrock to access S3 resources for training.

- **Hyperparameters**:
  - `epochCount`: Number of epochs for training. Adjust based on convergence and overfitting.
  - `batchSize`: Number of samples per batch. Tune this based on memory and training speed.
  - `learningRate`: Learning rate for training. Lower values can improve stability.

- **Data Configuration**:
  - `trainingDataConfig`: Specifies the S3 URI where the training dataset is stored.
  - `outputDataConfig`: Specifies the S3 URI where the fine-tuned model will be saved.

### Notes:
- Ensure the S3 bucket is accessible with the appropriate permissions.
- Monitor the job in the AWS console under Amazon Bedrock to track progress.

In [ ]:
# Define the job parameters
base_model_id = "cohere.command-light-text-v14:7:4k"
job_name = "cohere-medical-summary-finetune-job-v1" # Customizable: Name of the Finetuning job
model_name = "cohere-Summarizer-medical-Tuned-v1"

# Submit the fine-tuning job
bedrock.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=model_name,
    roleArn=role_arn,
    baseModelIdentifier=base_model_id,
    hyperParameters={
        "epochCount": "3", # Adjust based on convergence and overfitting
        "batchSize": "16", # Adjust based on memory availability and training speed
        "learningRate": "0.00005", # Adjust based on training stability and speed
    },
    trainingDataConfig={"s3Uri": f"s3://{bucket_name}/{s3_key}"},
    outputDataConfig={"s3Uri": f"s3://{bucket_name}/finetuned/"}
)

You can check the job status (next step) to make sure if it is finished or still being trained.

## Step 8: Monitor the Job Status

This step checks the status of the fine-tuning job submitted to Amazon Bedrock. Monitoring the job status is crucial for understanding its progress and determining when it has completed or if any issues have occurred.


In [ ]:
# Check the job status
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
print(f"Job status: {status}")

## Step 9: Perform Model Inference

To use the model for inference, you need to purchase "Provisioned Throughput." On Amazon Bedrock sidebar in your AWS console, go to "Custom Models" and then choose the "Models" tab, select the model you have trained, and then click on "Purchase Provisioned Throughput."



![Provisioned Throughput](images/provisioned-throughput.png)

Give the provisioned throughput a name, select a commitment term (you can choose "No Commitment" for testing), and then click "Purchase Provisioned Throughput." You will be able to see the estimated price as well. Once this is set up, you'll be able to use the model for inference.

![Commitment term](images/commitment-term.png)

Give the provisioned throughput a name, select a commitment term (you can choose "No Commitment" for testing), and then click "Purchase Provisioned Throughput." Once this is set up, you'll be able to use the model for inference. You can also see the estimated price for each commitment term.

![Alt text](images/provision.png)

To access your deployed model's endpoint, you'll need its ARN. Go to the "Provisioned Throughput" section under Inference in the sidebar. Select the name of your fine-tuned model, and on the new page, copy the ARN for use in the next step. Keep in mind that provisioning throughput may take a few minutes to complete.

![Custom Model ARN](images/custom-model-arn.png)

In the next step, we will make a request to the model for inference. Be sure to replace YOUR_MODEL_ARN with the ARN you copied in the previous step.

In [ ]:
# Initialize Bedrock runtime client
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=bedrock_region)

# Define a prompt for model inference
prompt = """
[doctor] Good morning, Mr. Smith. How have you been feeling since your last visit?  
[patient] Good morning, doctor. I've been okay overall, but I’ve been struggling with persistent fatigue and some dizziness.  
[doctor] I see. Is the dizziness occurring frequently or only under specific circumstances?  
[patient] It’s mostly when I stand up quickly or after I've been walking for a while.  
[doctor] Have you noticed any changes in your heart rate or shortness of breath during these episodes?  
[patient] No shortness of breath, but I do feel my heart racing sometimes.  

[doctor] How about your medications? Are you taking them as prescribed?  
[patient] Yes, but I missed a few doses of my beta-blocker last week due to travel.  
[doctor] That could explain some of the symptoms. I’ll need to check your blood pressure and do an EKG to assess your heart rhythm.  
[patient] Okay, doctor.  

[doctor] How has your diet been? Are you still following the low-sodium plan we discussed?  
[patient] I’ve been trying, but I’ve slipped up a bit during holidays with family meals.  
[doctor] I understand. We’ll reinforce that, as it’s critical for managing your hypertension.  
[patient] Yes, I’ll make sure to get back on track.  

[doctor] Let’s discuss the results from your last bloodwork. Your cholesterol levels were slightly elevated, and your hemoglobin A1c suggests borderline diabetes.  
[patient] I see. What does that mean for me?  
[doctor] It means we need to focus on dietary changes and consider starting a low-dose statin. I’ll also refer you to a nutritionist for better meal planning.  
[patient] That makes sense. Thank you, doctor.  

[doctor] Lastly, you mentioned experiencing more frequent leg swelling recently. Is that still a concern?  
[patient] Yes, especially after long days at work.  
[doctor] That could be a sign of fluid retention. I’ll adjust your diuretic dose and monitor your progress over the next two weeks.  
[patient] Thank you, doctor.  

[doctor] All right, let’s get those tests done and review everything at our next appointment. Do you have any other concerns?  
[patient] No, I think that’s all for now.  
[doctor] Great. See you in two weeks. 
"""

# Define the inference request body
body = {
    "prompt": prompt,
    "temperature": 0.5,
    "p": 0.9,
    "max_tokens": 80,
}

# Specify the ARN of the custom model
custom_model_arn = "YOUR_MODEL_ARN" #Put your model ARN here

# Invoke the custom model for inference
try:
    response = bedrock_runtime.invoke_model(
        modelId=custom_model_arn,
        body=json.dumps(body)
    )

    # Read and parse the response
    response_body = response['body'].read().decode('utf-8')
    result = json.loads(response_body)

    # Extract the summary from the response
    summary_text = result['generations'][0]['text']
    print("Extracted Summary:", summary_text)
except Exception as e:
    print(f"Error invoking model: {e}")

You can also test the inference directly from the Playground in the Amazon Bedrock console. To do this, navigate to Chat/Text under the Playground section, select your fine-tuned model, and enter your desired prompt.

![Playground](images/playground.png)

### Cleanup

<span style="color:red">To avoid incurring additional costs, please ensure that you **remove any provisioned throughput**.</span>

<span style="color:red">You can remove provisioned throughput by navigating to the **Provisioned Throughput** section from the sidebar in the Amazon Bedrock console. Select the active provisioned throughput and delete it.</span>